In [14]:
import csv
from gensim.models import Word2Vec
import pandas as pd
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout, ThresholdedReLU, Flatten
from keras.models import Sequential
from tensorflow.keras import Model
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import nltk
from nltk.util import ngrams
import seaborn as sns
import re
import math
from scipy.spatial.distance import euclidean, pdist, squareform
from sklearn.metrics.pairwise import pairwise_distances
!pip install hdbscan
import hdbscan
from sklearn.cluster import DBSCAN

from sklearn.datasets import make_blobs

tf.test.gpu_device_name() 

'/device:GPU:0'

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# downloaded = drive.CreateFile({'id':'1LAXYuJapp6u6shheA3ciIy67GrOQdlKs'}) 
# downloaded.GetContentFile('q2_revised.csv')  
df2 = pd.read_csv('/content/drive/Shared drives/Hackathon_new/q2_revised.csv', encoding='latin1')
# Dataset is now stored in a Pandas Dataframe
gt2 = pd.read_excel('/content/drive/Shared drives/Hackathon_new/q2_answer.xlsx')
alexa = pd.read_csv('/content/drive/Shared drives/Hackathon_new/alexa.csv', header=None, names=['dns'], usecols=[1])

words = pd.read_csv('/content/drive/Shared drives/Hackathon_new/google-10000-english.txt', sep=" ", header=None)
words.columns =["words"]

In [17]:
df2.head()

,No.,Time,Source,Destination,Protocol,Length,Text item,Transaction ID,Name,Time.1,Reply code,Response,Primary name server,Info
0,1,0.000000,183.213.21.48,166.111.8.29,DNS,73,â\234\223,0x2719,weilaigou.com,NaN,NaN,Message is a query,NaN,Standard query 0x2719 A weilaigou.com
1,2,0.044210,8.8.8.8,101.6.69.132,DNS,102,â\234\223,0x51ae,taobao.com,NaN,No error,Message is a response,NaN,Standard query response 0x51ae A taobao.com A ...
2,3,0.069832,8.8.8.8,101.6.128.60,DNS,68,â\234\223,0xf102,sohu.com,NaN,Server failure,Message is a response,NaN,Standard query response 0xf102 Server failure ...
3,4,0.090890,144.217.10.132,166.111.124.17,DNS,79,â\234\223,0xbde3,hrsa.gov,NaN,NaN,Message is a query,NaN,Standard query 0xbde3 ANY hrsa.gov OPT
4,5,0.147890,183.173.94.53,8.8.8.8,DNS,70,â\234\223,0xb6ce,wahom.site,NaN,NaN,Message is a query,NaN,Standard query 0xb6ce A wahom.site


In [18]:

gt2[gt2.domain.str.contains("?", regex=False)]



,domain,family
10992,????.cc,10


In [0]:
gt2.drop(10992, inplace=True)

In [20]:
gt2

,domain,family
0,z54f71fcb61e49158f3648236556b04eb1.so,1
1,l31f5dcf7f398d2b94cd9155ee392c701b.so,1
2,d442a48406424819e686a5d1fde7c602ea.so,1
3,zba909e91846ce27aff7902f9228ee40ea.so,1
4,p8f5889e854374cf8e70e91f456807f83f.so,1
...,...,...
11090,gdwrhiivyt.cc,10
11091,hyirnrnetku.cc,10
11092,isauvvu.cc,10
11093,vddiuiygf.cc,10


In [0]:
df2.drop_duplicates(keep='first', inplace=True)
gt2.drop_duplicates(subset='domain',keep='last', inplace=True) #ws more likely to be in family 2
df2 = df2.rename(columns={'Name': 'domain'})


In [22]:
dfmal = pd.merge(df2, gt2, how='inner', on=['domain'])
dfmal



,No.,Time,Source,Destination,Protocol,Length,Text item,Transaction ID,domain,Time.1,Reply code,Response,Primary name server,Info,family
0,227,1.775349,166.111.8.28,193.232.156.17,DNS,83,â\234\223,0xf6f7,rosnastil.ru,NaN,NaN,Message is a query,NaN,Standard query 0xf6f7 MX rosnastil.ru OPT,4
1,729770,3802.363534,166.111.8.28,193.232.156.17,DNS,83,â\234\223,0xdd81,rosnastil.ru,NaN,NaN,Message is a query,NaN,Standard query 0xdd81 MX rosnastil.ru OPT,4
2,729789,3802.448643,166.111.8.28,95.211.92.15,DNS,83,â\234\223,0x0649,rosnastil.ru,NaN,NaN,Message is a query,NaN,Standard query 0x0649 MX rosnastil.ru OPT,4
3,729917,3803.248868,166.111.8.28,5.254.122.130,DNS,83,â\234\223,0x4863,rosnastil.ru,NaN,NaN,Message is a query,NaN,Standard query 0x4863 MX rosnastil.ru OPT,4
4,729967,3803.592610,5.254.122.130,166.111.8.28,DNS,247,â\234\223,0x4863,rosnastil.ru,0.343742,No error,Message is a response,NaN,Standard query response 0x4863 MX rosnastil.ru...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75246,5253090,19712.936867,16.69.196.83,44.192.244.210,DNS,141,â\234\223,0xfafd,ozndemknlvnzens.biz,0.032079,No such name,Message is a response,a.gtld.biz,Standard query response 0xfafd No such name A ...,3
75247,5253218,19713.478582,166.111.8.28,217.160.82.43,DNS,82,â\234\223,0x5ba4,laurich.org,NaN,NaN,Message is a query,NaN,Standard query 0x5ba4 MX laurich.org OPT,8
75248,5253274,19713.714548,217.160.82.43,166.111.8.28,DNS,139,â\234\223,0x5ba4,laurich.org,0.235966,No error,Message is a response,NaN,Standard query response 0x5ba4 MX laurich.org ...,8
75249,5256606,19727.335650,44.192.244.210,16.69.196.83,DNS,78,â\234\223,0x76a0,sslqldygoicpff.org,NaN,NaN,Message is a query,NaN,Standard query 0x76a0 A sslqldygoicpff.org,8


In [0]:
#examine relationships between IP and family

In [0]:
domains = list(gt2.domain)

In [35]:
ips = list(set(dfmal.loc[dfmal['Response'] == "Message is a query"].Source))
print(ips)
print(len(ips)) #335 infected IPs. 

['123.114.120.72', '101.28.115.158', '166.111.50.21', '205.215.81.221', '211.68.127.2', '112.22.90.65', '171.9.72.173', '113.121.209.134', '219.131.11.66', '113.85.97.158', '39.90.118.154', '139.33.104.134', '58.48.128.15', '113.85.99.84', '37.143.23.6', '89.89.143.48', '123.114.126.143', '132.110.76.173', '110.85.69.156', '120.229.94.103', '195.96.148.131', '60.183.65.41', '58.37.200.156', '166.111.8.28', '223.87.210.125', '61.48.211.9', '119.39.248.121', '27.11.246.1', '198.46.140.162', '59.66.57.58', '211.87.232.199', '39.76.69.246', '221.196.254.132', '183.212.187.70', '166.111.80.237', '243.162.172.198', '101.6.64.53', '91.4.216.203', '183.198.217.113', '59.66.37.70', '14.211.252.201', '6.190.219.69', '161.76.32.19', '14.120.55.61', '222.29.47.196', '223.74.34.2', '182.90.209.201', '211.68.127.48', '106.114.167.50', '183.172.72.122', '58.22.52.190', '202.113.176.174', '59.66.32.178', '124.78.37.4', '112.32.82.219', '166.111.141.136', '112.10.170.49', '139.199.23.110', '112.51.32.1

In [0]:
#construct 2d matrix. rows are domains. columns are IP address
def getmatrix():
  dic = {}
  for i in ips:
    tmp = []
    domlist = set((dfmal.loc[dfmal['Response'] == "Message is a query"].loc[dfmal.loc[dfmal['Response'] == "Message is a query"]['Source'] == i].drop_duplicates(subset='domain', keep="first")).domain) #only counting the first query
    for d in domains:
      if d in domlist:
        tmp.append(1) #1 means ip queried that domain
      else:
        tmp.append(0)
    dic[i] = tmp #ex. dic[166.111.4.188] = [0,0,1,0,1,0] these will correspond to domains that ip queried
  return dic
  


In [0]:
matrix = getmatrix()
#matrix

In [40]:
1 in matrix['166.111.4.188']

True

In [43]:
DF_var = pd.DataFrame.from_dict(matrix)
DF_var.index = domains
DF_var #matrix representation of domains and IPs

,123.114.120.72,101.28.115.158,166.111.50.21,205.215.81.221,211.68.127.2,112.22.90.65,171.9.72.173,113.121.209.134,219.131.11.66,113.85.97.158,39.90.118.154,139.33.104.134,58.48.128.15,113.85.99.84,37.143.23.6,89.89.143.48,123.114.126.143,132.110.76.173,110.85.69.156,120.229.94.103,195.96.148.131,60.183.65.41,58.37.200.156,166.111.8.28,223.87.210.125,61.48.211.9,119.39.248.121,27.11.246.1,198.46.140.162,59.66.57.58,211.87.232.199,39.76.69.246,221.196.254.132,183.212.187.70,166.111.80.237,243.162.172.198,101.6.64.53,91.4.216.203,183.198.217.113,59.66.37.70,...,202.107.141.77,183.172.184.77,166.111.32.133,132.201.172.28,183.172.88.59,166.111.68.133,213.138.78.224,61.148.245.59,36.57.160.18,213.99.215.146,111.35.18.241,18.37.227.134,132.177.237.57,182.114.146.29,183.172.144.62,115.210.70.202,223.104.1.105,166.111.24.24,59.66.32.99,111.18.176.57,207.208.6.4,223.11.214.46,117.30.152.81,46.146.152.237,114.89.41.40,166.111.186.49,183.172.128.89,58.219.62.252,183.40.19.93,153.3.76.134,113.250.222.95,118.195.207.193,39.91.49.17,166.111.156.5,220.173.153.187,183.206.131.38,162.105.187.205,1.189.57.199,111.35.219.142,38.99.249.103
z54f71fcb61e49158f3648236556b04eb1.so,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
l31f5dcf7f398d2b94cd9155ee392c701b.so,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
d442a48406424819e686a5d1fde7c602ea.so,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
zba909e91846ce27aff7902f9228ee40ea.so,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
p8f5889e854374cf8e70e91f456807f83f.so,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gdwrhiivyt.cc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
hyirnrnetku.cc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
isauvvu.cc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
vddiuiygf.cc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#DF_var.to_pickle("/content/drive/Shared drives/Hackathon_new/DF_var.pkl")


In [47]:
#y is family
y = list(gt2.family)
dic = {}
for i in list(DF_var):
  d2 = {}
  domainlist = list(DF_var[i])
  for d in range(len(domainlist)):
    if domainlist[d] == 1:
      if y[d] not in d2:
        d2[y[d]] = 1
      else:
        d2[y[d]] += 1
  print(i, d2)
  dic[str(i)] = d2

123.114.120.72 {6: 1}
101.28.115.158 {6: 1}
166.111.50.21 {2: 1, 8: 5}
205.215.81.221 {3: 19, 5: 19, 6: 12, 8: 17}
211.68.127.2 {8: 1}
112.22.90.65 {8: 1}
171.9.72.173 {8: 1}
113.121.209.134 {6: 1}
219.131.11.66 {2: 1, 6: 1}
113.85.97.158 {8: 1}
39.90.118.154 {5: 1, 6: 2, 8: 4}
139.33.104.134 {2: 20, 3: 18, 5: 19, 6: 13, 8: 21, 10: 12}
58.48.128.15 {6: 1}
113.85.99.84 {4: 1, 8: 1}
37.143.23.6 {2: 13, 3: 31, 5: 27, 6: 26, 8: 32}
89.89.143.48 {2: 18, 3: 15, 5: 15, 6: 13, 8: 19, 10: 12}
123.114.126.143 {6: 1}
132.110.76.173 {1: 165, 2: 42}
110.85.69.156 {6: 3, 8: 2}
120.229.94.103 {8: 1}
195.96.148.131 {2: 12, 3: 17, 5: 12, 6: 16, 8: 22, 10: 21}
60.183.65.41 {6: 1}
58.37.200.156 {6: 1}
166.111.8.28 {2: 74, 3: 53, 4: 246, 5: 154, 6: 815, 7: 9, 8: 1154, 9: 109, 10: 141}
223.87.210.125 {6: 1}
61.48.211.9 {6: 1}
119.39.248.121 {6: 2, 10: 1}
27.11.246.1 {2: 1}
198.46.140.162 {3: 22, 5: 23, 6: 14, 8: 22}
59.66.57.58 {6: 1}
211.87.232.199 {6: 1, 8: 1}
39.76.69.246 {6: 1}
221.196.254.132 {8: 1}
1

In [46]:
print(dic) #we observe that some hosts are infected with multiple families of DGAs. this makes our task harder but it still is useful

{'123.114.120.72': {6: 1}, '101.28.115.158': {6: 1}, '166.111.50.21': {2: 1, 8: 5}, '205.215.81.221': {3: 19, 5: 19, 6: 12, 8: 17}, '211.68.127.2': {8: 1}, '112.22.90.65': {8: 1}, '171.9.72.173': {8: 1}, '113.121.209.134': {6: 1}, '219.131.11.66': {2: 1, 6: 1}, '113.85.97.158': {8: 1}, '39.90.118.154': {5: 1, 6: 2, 8: 4}, '139.33.104.134': {2: 20, 3: 18, 5: 19, 6: 13, 8: 21, 10: 12}, '58.48.128.15': {6: 1}, '113.85.99.84': {4: 1, 8: 1}, '37.143.23.6': {2: 13, 3: 31, 5: 27, 6: 26, 8: 32}, '89.89.143.48': {2: 18, 3: 15, 5: 15, 6: 13, 8: 19, 10: 12}, '123.114.126.143': {6: 1}, '132.110.76.173': {1: 165, 2: 42}, '110.85.69.156': {6: 3, 8: 2}, '120.229.94.103': {8: 1}, '195.96.148.131': {2: 12, 3: 17, 5: 12, 6: 16, 8: 22, 10: 21}, '60.183.65.41': {6: 1}, '58.37.200.156': {6: 1}, '166.111.8.28': {2: 74, 3: 53, 4: 246, 5: 154, 6: 815, 7: 9, 8: 1154, 9: 109, 10: 141}, '223.87.210.125': {6: 1}, '61.48.211.9': {6: 1}, '119.39.248.121': {6: 2, 10: 1}, '27.11.246.1': {2: 1}, '198.46.140.162': {3: 